<p style="font-size:260%;line-height:1.5">Generate SEDs of compact sources from the Planck Catalog of Compact Sources </p>

<p style="font-size:140%;line-height:1.2">
    Here I develop a set of functions to read the Planck PCCS and derive the SEDs of compact sources that are seen at all frequencies
    </p>

# Import and functions

## Imports and common data

In [ ]:
import datetime                  as dt
import pytz
import numpy                     as np       # Numerical Python
import pylab                     as pl       # Matplotlib
from astropy.io.fits import open as fitsOpen # For FITS files
import pickle
from scipy import interpolate

In [ ]:
freqs    = ['030','044','070','100','143','217','353']
freqs_ex = ['100','143','217','353']
altnames = {\
           'Crab'       : '184.5-05.8',\
           'RCW28'      : '267.9-01.1',\
           'Orion'      : '209.0-19.4'}

In [ ]:
catalog_dir = '/home/daniele/Documents/QUBIC/operations/TD/operations_plan/catalogs/'

## Functions

### Check if source is in catalog

In [ ]:
def isincatalog(source, catalog):
    
    freqs    = ['030','044','070','100','143','217','353']
    
    # Make list of all sources
    allsources = []
    for f in freqs:
        allsources = allsources + list(catalog[f].keys())
    allsources = list(set(allsources))

    # Check if source exists in catalog
    if source in list(altnames.keys()):
        return True, altnames[source]
    elif source in allsources:
        return True, source
    else:
        return False, ''       

In [ ]:
isincatalog('Crab',catalog)

### Build catalog from PCCS

In [ ]:
def build_catalog(freqs = freqs, freqs_ex = freqs_ex, excluded = True):
    '''
    This function builds a dictionary containing the main parameters of the compact sources
    contained in the PCCS
        
    Input
    freqs       - LIST  - List of frequencies in the catalog (Default 
                          ['030','044','070','100','143','217','353'])
    freqs_ex     - LIST - List of frequencies of excluded catalogs (Default
                          ['100','143','217','353'])
    excluded     - BOOL - Whether to include the excluded catalogs (Default: True)
        
    Output
    catalog      - DICT - Dictionary containing the data
    '''
    
    catalog = {}

    # Read normal catalogs
    global_namelist = []
    for f in freqs:
        
        print('Building catalog at %s GHz from PCCS2' % f)
        catalog[f] = {}
        fname = '%sCOM_PCCS_%s_R2.01.fits' % (catalog_dir, f)
        fd = fitsOpen(fname, "readonly")
        names    = fd[1].data['NAME    ']
        ras      = fd[1].data['RA      ']
        decs     = fd[1].data['DEC     ']
        gLons    = fd[1].data['GLON    ']
        gLats    = fd[1].data['GLAT    ']
        detFluxs = fd[1].data['DETFLUX ']
        ps       = fd[1].data['P       ']
        fd.close()
        for name, ra, dec, gLon, gLat, detFlux, p in zip (names, ras, decs, gLons, gLats, detFluxs, ps):
            if f == freqs[0]:
                # If we are scanning the first frequency then define names based on GLON and GLAT
                # Rounded to 1 decimal place
                
                new_name = build_name(name)
                
                global_namelist.append(new_name)

            else:
                # For other frequencies see if each source is close enough to be one of the first frequency
                # set. In this case use the name already used in the first set, otherwise define new name 
                # based on rounded GLON GLAT
                
                new_name = build_name(name)
                
                source_exist, new_name = duplicate_source(new_name, global_namelist)
                
                if source_exist == False:
                    global_namelist.append(new_name)
                
            catalog[f][new_name]    = {}
            catalog[f][new_name]['RA']      = float(ra)
            catalog[f][new_name]['DEC']     = float(dec)
            catalog[f][new_name]['GLON']    = float(gLon)
            catalog[f][new_name]['GLAT']    = float(gLat)
            catalog[f][new_name]['DETFLUX'] = float(detFlux)
            catalog[f][new_name]['PFLUX']   = float(p)
            catalog[f][new_name]['ALTNAME'] = ''
        
    if excluded:

        # Read excluded catalogs
        for f in freqs_ex:
            print('Building catalog at %s GHz from PCCS2E' % f)
            fname = '%sCOM_PCCS_%s-excluded_R2.01.fits' % (catalog_dir, f)
            fd = fitsOpen(fname, "readonly")
            names    = fd[1].data['NAME    ']
            ras      = fd[1].data['RA      ']
            decs     = fd[1].data['DEC     ']
            gLons    = fd[1].data['GLON    ']
            gLats    = fd[1].data['GLAT    ']
            detFluxs = fd[1].data['DETFLUX ']
            ps       = fd[1].data['P       ']
            fd.close()
            for name, ra, dec, gLon, gLat, detFlux, p in zip (names, ras, decs, gLons, gLats, detFluxs, ps):

                new_name = build_name(name)

                source_exist, new_name = duplicate_source(new_name, global_namelist)
                
                if source_exist == False:
                    global_namelist.append(new_name)
 
                catalog[f][new_name]    = {}
                catalog[f][new_name]['RA']      = float(ra)
                catalog[f][new_name]['DEC']     = float(dec)
                catalog[f][new_name]['GLON']    = float(gLon)
                catalog[f][new_name]['GLAT']    = float(gLat)
                catalog[f][new_name]['DETFLUX'] = float(detFlux)
                catalog[f][new_name]['PFLUX']   = float(p)
                catalog[f][new_name]['ALTNAME'] = ''

    return catalog

In [ ]:
def build_name(name):
    '''
    This function builds a source name from the PCCS name by rounding l and b to the first decimal place
        
    Input
    name         - STRING - source name as defined in the PCCS (Glll.ll±bb.bb)
        
    Output
    new_name     - STRING - source new name defined as lll.l±bb.b
    '''
    
    name_l = np.round(float(name[-12:-6]),1)
    str_l  = '%05.1f' % name_l
    sign_b = name[-6]
    name_b = np.round(float(name[-5:]),1)
    str_b = '%04.1f' % name_b
    new_name = str_l.rjust(5,'0') + sign_b + str_b.rjust(4,'0')
    
    return new_name

In [ ]:
def duplicate_source(name, global_namelist, threshold = 0.1):
    '''
    This function finds if a given source is a duplicate of others already found in catalogs
    relative to other frequencies by checking the distance in GLON and GLAT
        
    Input
    name                   - STRING       - source name modified by the catalog building routine
                                            (Glll.l±bb.b)
    global_namelist        - LIST         - list of modified names of sources already loaded from other 
                                            frequency catalogs
    threshold              - FLOAT        - maximum distance in degrees to decide whether two sources  
                                            coincide (Default threshold = 0.1)
    Output
    isduplicate, new_name  - BOOL, STRING - whether a duplicate has been found, new name
    '''
    name_l = float(name[0:5])
    name_b = float(name[-5:])
    
    for item in global_namelist:
        ex_l = float(item[0:5])
        ex_b = float(item[-5:])
        
        if (np.abs(name_l - ex_l) <= threshold) and (np.abs(name_b - ex_b) <= threshold):
            # In this case we have the same source, return True and the name
            return True, item
    
    return False, name    

### Build SEDs

#### SEDs of common sources

In [ ]:
def build_sed_allfreqs(catalog, freqs = freqs):
    '''
    This function builds the SED of the sources in the catalog using data across frequencies specified
    in freqs
        
    Input
    catalog      - DICT - The dictionary with the source catalog

    freqs        - LIST - List of frequencies (Default ['030', '044', '070', 100','143','217','353'])
        
    Output
    SED          - DICT - Dictionary containing the SED (frequencies, measured I_flux, measured P_flux
                          4th order polinomial fits to measured I_flux and P_flux
    '''    
    # Build common set of sources
    inters = ''
    for f in freqs:
        inters = inters + ('set(catalog["%s"].keys()) & ' % f)
    inters = 'list(' + inters[0:-2] + ')'
    common_sources = eval(inters)
    
    flist = np.array(list(map(float,freqs)))
    
    SED = {}
    
    for source in common_sources:
        SED[source] = {}
        i_flux    = np.array([catalog[f][source]['DETFLUX'] for f in freqs])
        p_flux    = np.array([catalog[f][source]['PFLUX'] for f in freqs])
        sed_i_fit = np.polyfit(flist, i_flux,4)
        sed_p_fit = np.polyfit(flist, p_flux,4)
        SED[source]['freq']   = flist
        SED[source]['i_flux'] = i_flux
        SED[source]['p_flux'] = p_flux
        SED[source]['i_fit']  = sed_i_fit
        SED[source]['p_fit']  = sed_p_fit

    return SED


#### SED of a given source

In [ ]:
def build_sed(source, catalog, plot = False, polyfit = 3):
    '''
    This function builds the SED of a given source
        
    Input
    source       - STRING - The source name, either in the stanard lll.ll±bb.bb format or in the common 
                            name format if a translation is available (e.g. Crab)

    catalog      - DICT - The dictionary with the source catalog

    plot         - BOOL - Whether to plot intensity and polarized fluxes. Default: False

    polyfit      - INT  - Order of the polynomial fit. Default: 3

    Output
    SED          - DICT - Dictionary containing the SED (frequencies, measured I_flux, measured P_flux
                          4th order polinomial fits to measured I_flux and P_flux
    '''    
    
    # Check if source is in catalog
    exists, sourcename = isincatalog(source, catalog)
    
    if not exists:
        print('Source %s is not in catalog' % source)
        return -1
    
    # Get the number of frequencies at which we have data
    s_freqs = source2freqs(source, catalog)
    if len(s_freqs) <= 1:
        print('Not enough frequencies to build a SED')
        return -1

    flist = np.array(list(map(float,s_freqs)))

    SED = {}
    
    SED[sourcename] = {}
    
    i_flux    = np.array([catalog[f][sourcename]['DETFLUX'] for f in s_freqs])
    p_flux    = np.array([catalog[f][sourcename]['PFLUX']   for f in s_freqs])
    sed_i_fit = np.polyfit(flist, i_flux, polyfit)
    sed_p_fit = np.polyfit(flist, p_flux, polyfit)
    SED[sourcename]['freq']   = flist
    SED[sourcename]['i_flux'] = i_flux
    SED[sourcename]['p_flux'] = p_flux
    SED[sourcename]['i_fit']  = sed_i_fit
    SED[sourcename]['p_fit']  = sed_p_fit
    
    if plot:
        newfreq = np.arange(flist[0], flist[-1] + 1, 1)
        fi = np.poly1d(SED[sourcename]['i_fit'])
        fp = np.poly1d(SED[sourcename]['p_fit'])
        
        pl.figure(figsize = (13,7))
        
        # Intensity plot
        pl.subplot(121)
        pl.plot(SED[sourcename]['freq'],SED[sourcename]['i_flux'],'.')
        pl.plot(newfreq, fi(newfreq))
        pl.xlabel('Frequency [GHz]')
        pl.ylabel('Flux [mJy]')
        pl.title('%s - Intensity flux' % source)
        
        # Polarization plot
        pl.subplot(122)
        pl.plot(SED[sourcename]['freq'],SED[sourcename]['p_flux'],'.')
        pl.plot(newfreq, fp(newfreq))
        pl.xlabel('Frequency [GHz]')
        pl.ylabel('Flux [mJy]')
        pl.title('%s - Polarized flux' % source)
        

    return SED


In [ ]:
sed = build_sed('Crab', catalog, plot = True, polyfit = 3)

### Translate from common source name to catalog name

In [ ]:
def name2cat(name, altnames):

    if name not in list(altnames.keys()):
        print('Name %s not known' % name)
        return -1
    
    return altnames[name]

### Return the frequencies of a given source name 

In [ ]:
def source2freqs(source, catalog, altnames = altnames):
    '''
    This function return the list of frequencies in the catalog given a certain source
        
    Input
    source       - STRING - The source name, either in the stanard lll.ll±bb.bb format or in the common 
                            name format if a translation is available (e.g. Crab)

    catalog      - DICT   - The PCCS in Qubic format
        
    altnames     - DICT   - The correspondence between common name and catalog standard name (Defaults to 
                            altnames defined at the top of the notebook)
 
    Output
    freqlist     - LIST - List of frequencies where a source is found
    '''        
    exists, sourcename = isincatalog(source, catalog)
    
    if not exists:
        print('Source %s is not in catalog' % source)
        return -1        
    
    isinfreq = [sourcename in list(catalog[f].keys()) for f in freqs]
    
    return [freqs[i] for i in list(np.where(isinfreq)[0])]

In [ ]:
source2freqs('Orion',catalog)

# Catalog

## Build and save catalog to disk

In [ ]:
catalog = build_catalog()

In [ ]:
catalog_file = catalog_dir + 'qubic_pccs2.pickle'
with open(catalog_file, 'wb') as handle:
    pickle.dump(catalog, handle)

## Load catalog from disk

In [ ]:
catalog_file = catalog_dir + 'qubic_pccs2.pickle'
with open(catalog_file, 'rb') as handle:
    catalog = pickle.load(handle)

In [ ]:
catalog['070']['184.5-05.8']

In [ ]:
catalog['143']['184.5-05.8']

In [ ]:
catalog['143'][name2cat('RCW28',altnames)]

In [ ]:
catalog['143'].keys()

# SED

## Build SED and save to disk

In [ ]:
SED = build_sed(catalog)

In [ ]:
sed_file = catalog_dir + 'qubic_pccs2_sed.pickle'
with open(sed_file, 'wb') as handle:
    pickle.dump(SED, handle)

## Load SED, plot data for a source

In [ ]:
sed_file = catalog_dir + 'qubic_pccs2_sed.pickle'
with open(sed_file, 'rb') as handle:
    SED = pickle.load(handle)

<p style="font-size:140%;line-height:1.2">
    Let's see the SED in I and P of the Crab
</p>

In [ ]:
mysed = SED[name2cat('Orion')]

In [ ]:
mysed.keys()

In [ ]:
newfreq = np.arange(30,354,1)
f = np.poly1d(mysed['i_fit'])
pl.plot(mysed['freq'],mysed['i_flux'],'.')
pl.plot(newfreq, f(newfreq))
pl.xlabel('Frequency [GHz]')
pl.ylabel('Flux [mJy]')
pl.title('Crab - Intensity flux')

In [ ]:
newfreq = np.arange(30,354,1)
f = np.poly1d(mysed['p_fit'])
pl.plot(mysed['freq'],mysed['p_flux'],'.')
pl.plot(newfreq, f(newfreq))
pl.xlabel('Frequency [GHz]')
pl.ylabel('Flux [mJy]')
pl.title('Crab - Polarized flux')